In [7]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import Dataset, DataLoader

# 1. CSV 파일 로드
abbreviations = pd.read_csv('./data/abbreviations.csv')  # 약어 CSV 파일
simplified = pd.read_csv('./data/simplified.csv')  # 용어 설명 CSV 파일

# 2. 한글/영어 약어 자동 판별 함수
def is_korean(text):
    """
    텍스트가 한글로 구성되어 있는지 확인합니다.
    """
    return bool(re.search(r'[가-힣]', text))

# 3. 약어 대체 함수
def replace_abbreviations(text, abbreviations):
    """
    한글 약어는 전체 용어로 바로 변환하고, 영어 약어는 괄호로 풀어쓴 용어를 추가합니다.
    """
    for _, row in abbreviations.iterrows():
        abbreviation = row["Abbreviation"]
        full_term = row["Full Term"]

        if is_korean(abbreviation):
            # 한글 약어는 바로 전체 용어로 변환
            text = re.sub(rf'\b{re.escape(abbreviation)}\b', full_term, text)
        else:
            # 영어 약어는 괄호로 풀어쓴 용어 추가
            text = re.sub(rf'\b{re.escape(abbreviation)}\b', f"{abbreviation}({full_term})", text)
    
    return text

# 4. 어려운 용어 변환 함수
def simplify_terms(text, simplified):
    """
    어려운 용어를 정의와 영향 설명으로 대체합니다.
    """
    for _, row in simplified.iterrows():
        term_pattern = rf'\b{re.escape(row["Term"])}\b'
        definition = f"{row['Definition']} ({row['Impact']})"
        text = re.sub(term_pattern, definition, text)
    return text

# 5. 데이터셋 클래스 정의
class ArticleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
        }

# 6. 경제 기사 처리 함수
def process_article_data(input_file, abbreviations, simplified):
    """
    기사의 약어를 대체하고 용어를 변환하여 처리합니다.
    """
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # 약어 대체 및 용어 변환
    text = replace_abbreviations(text, abbreviations)
    text = simplify_terms(text, simplified)

    return text

# 7. BERT 모델 로드 및 초기화 (Masked Language Model)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# 8. 경제 기사 처리
input_file = './data/input.txt'  # 처리할 경제 기사 파일
processed_text = process_article_data(input_file, abbreviations, simplified)

# 9. 텍스트 예측을 위한 데이터셋 생성
data = [processed_text]  # 예제 데이터 생성
dataset = ArticleDataset(data, tokenizer)

data_loader = DataLoader(dataset, batch_size=1)

# 10. 변환된 텍스트 출력
print("변환된 텍스트:")
print(processed_text)


/home/u4009/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


변환된 텍스트:
한투운용은 ACE KRX( 한국거래소) 금현물 ETF의 순자산액이 5000억원을 돌파했다고 25일 밝혔다. 해당 ETF의 빠른 순자산 증가는 개인투자자들의 매수세와 우수한 수익률 덕으로 풀이된다. 최근 지속되는 중동 전쟁과 악화하는 러시아와 우크라이나 전쟁으로 안전자산인 금에 대한 개인 투자자들의 관심이 쏠린 것으로 해석된다.
개인연금 및 퇴직연금에서의 자금 유입도 컸다.
안전자산인 금현물에 장기 투자하며 세액공제 혜택을 받고자 하는 투자자들이 늘어난 것으로 분석된다.

